In [9]:
# ------------------------------------
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
# ------------------------------------

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from pathlib import Path
from dotenv import load_dotenv

from azure.ai.agents.models import OpenApiTool, OpenApiAnonymousAuthDetails
from azure.identity import DefaultAzureCredential
import jsonref

load_dotenv()
AIFOUNDRY_AGENT_PROJECT_NAME = os.getenv("AIFOUNDRY_AGENT_PROJECT_NAME")
AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME")

In [10]:
project_client = AIProjectClient(
    endpoint=AIFOUNDRY_AGENT_PROJECT_NAME,
    credential=DefaultAzureCredential(),
)


agents_client = project_client.agents

In [11]:
import os

openapi_path = './OpenAPI specs/weather_openapi.json'
if not os.path.exists(openapi_path):
    raise FileNotFoundError(f"OpenAPI spec file not found at: {openapi_path}")

with open(openapi_path, 'r') as f:
    openapi_spec = jsonref.loads(f.read())

# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiAnonymousAuthDetails()

# Initialize agent OpenAPI tool using the read in OpenAPI spec
openapi = OpenApiTool(name="get_weather", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth)

In [12]:

# agent = project_client.agents.create_agent(
#     model="gpt-4o",
#     name="my-assistant",
#     instructions="You are a helpful assistant",
#     tools=openapi.definitions
# )


agent = agents_client.create_agent(
    model=AIFOUNDRY_AGENT_MODEL_DEPLOYMENT_NAME,
    name="my-agent",
    instructions="You are helpful agent",
    tools=openapi.definitions
)
# [END upload_file_and_create_agent_with_code_interpreter]
print(f"Created agent, agent ID: {agent.id}")


thread = agents_client.threads.create()
print(f"Created thread, thread ID: {thread.id}")

# Create a message
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather in Seattle?",
)
print(f"Created message, ID: {message.id}")

run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# [START get_messages_and_save_files]
messages = agents_client.messages.list(thread_id=thread.id)
print(f"Messages: {messages}")

# [START get_messages_and_save_files]
messages = agents_client.messages.list(thread_id=thread.id)
print(f"Messages: {messages}")

last_msg = agents_client.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

agents_client.delete_agent(agent.id)
print("Deleted agent")




Created agent, agent ID: asst_fB662pbeRoaZB96pIKFAeRSh
Created thread, thread ID: thread_07FBUN7JgRrpHuLPpfpuBzlX
Created message, ID: msg_P75Vrl2hsDjfxJpzlowcBNnH
Run finished with status: completed
Messages: <iterator object azure.core.paging.ItemPaged at 0x17f3d0e3f10>
Messages: <iterator object azure.core.paging.ItemPaged at 0x17f3d0e2d10>
Last Message: The current weather in Seattle is:

- **Condition:** Partly cloudy
- **Temperature:** 17°C (63°F)
- **Feels Like:** 17°C (63°F)
- **Humidity:** 84%
- **Wind:** North at 5 km/h (3 mph)
- **Visibility:** 16 km (9 miles)
- **Pressure:** 1017 hPa
- **Precipitation:** None reported

Let me know if you want a forecast for the next few days!
Deleted agent
